# Calibrating OGGM Mass Balance with CryoSat-2 Observations

If necessary, install the DTCG API with:

```
!pip install --upgrade git+https://github.com/DTC-Glaciers/dtcg
```

In a cell below.

In [ ]:
import os

import dtcg.integration.oggm_bindings as oggm_bindings
import matplotlib.pyplot as plt
import numpy as np
from oggm.core import massbalance
from datetime import datetime, timezone

In [ ]:
rgi_ids = ["RGI60-06.00377"]    #Bruarjökull, Iceland

## Initialise OGGM

In [ ]:
# DTCG OGGM binding for CryoTempo
dtcg_oggm = oggm_bindings.BindingsCryotempo()

# Initialise OGGM
dtcg_oggm.init_oggm()
gdir = dtcg_oggm.get_glacier_directories(rgi_ids = rgi_ids, prepro_level=4, prepro_border=80)[0]
dtcg_oggm.get_glacier_data(gdirs=[gdir])
dtcg_oggm.set_flowlines(gdir)

## Extract the Level 1 Datacube with CryoSat observations

DTC-Glaciers starts with glacier-domain data from the [OGGM shop](https://docs.oggm.org/en/stable/shop.html) (sourced from multiple providers) and packages it into a datacube. <br>
The `get_eolis_data` method retrieves this datacube and adds four [CryoTEMPO-EOLIS](https://cryotempo-eolis.org/) variables derived from CryoSat-2 observations.

| Variable | Dims | Description |
|---|---|---|
| `eolis_gridded_elevation_change` | `(t, y, x)` | Time series of spatial maps of elevation change since **January 2011**. |
| `eolis_gridded_elevation_change_sigma` | `(t, y, x)` | Uncertainty (σ) for the gridded elevation-change maps. |
| `eolis_elevation_change_timeseries` | `(t)` | Spatially aggregated 1D elevation-change series since **January 2011**. |
| `eolis_elevation_change_sigma_timeseries` | `(t)` | Uncertainty (σ) for the aggregated series. |


In [ ]:
gdir, datacube_handler = dtcg_oggm.get_eolis_data(gdir)
datacube = datacube_handler.get_layer("L1")

# display L1 datacube
datacube

Let visualise these new datasets!

In [ ]:
import xarray as xr

# decode for nice plotting (we dont do this during processing as it alters the metadata)
datacube_decoded = xr.decode_cf(datacube)

fig, axes = plt.subplots(ncols=2, nrows=1, figsize=(15, 5))
datacube_decoded.eolis_gridded_elevation_change.isel(t=-1).plot(ax=axes[0])
datacube_decoded.eolis_gridded_elevation_change_sigma.isel(t=-1).plot(ax=axes[1])
for ax in axes:
    ax.set_aspect('equal')
plt.suptitle("EOLIS Gridded Elevation Change (last time step shown) \n Glacier: {} RGI-ID: {}".format(gdir.name, gdir.rgi_id),
             fontweight='bold')
plt.tight_layout()
plt.show()

fig, ax = plt.subplots(figsize=(15, 5))
datacube_decoded.eolis_elevation_change_timeseries.plot(ax=ax)
ax.fill_between(
    datacube_decoded.t,
    datacube_decoded.eolis_elevation_change_timeseries - datacube_decoded.eolis_elevation_change_sigma_timeseries,
    datacube_decoded.eolis_elevation_change_timeseries + datacube_decoded.eolis_elevation_change_sigma_timeseries,
    alpha=0.5
)
ax.set_title("EOLIS Elevation Change Time Series \n Glacier: {} RGI-ID: {}".format(gdir.name, gdir.rgi_id),
             fontweight='bold')
ax.set_xlabel("Date")
plt.show()

## Use observations to calibrate OGGM

We convert the CryoSat-2 elevation-change time series ($\Delta h$) into a specific mass-balance rate ($\mathrm{dmdtda}$) over a chosen period, then compare with the geodetic reference from Hugonnet et al. (2021) retrieved from the OGGM shop.

**Steps**

1. **Compute elevation change over the period**  
   $$\Delta h = h(t_{\mathrm{end}}) - h(t_{\mathrm{start}})$$

2. **Convert to meters water-equivalent per year** (to match Hugonnet units) using a bulk density $\rho = 850\ \mathrm{kg\,m^{-3}}$ and period length $\Delta t$ in years (the factor 1000 converts $\mathrm{kg\ m^{-2}}$ to $\mathrm{m\ w.e.}$):
   $$
   \mathrm{dmdtda}\;[\mathrm{m\ w.e.\ yr^{-1}}]
   = \frac{\Delta h\,\rho}{\Delta t \cdot 1000}
   $$

> **Note:** CryoTEMPO-EOLIS uncertainties are not yet propagated in this conversion.

**Reference data**

- Geodetic mass balance from **Hugonnet et al. (2021)**, pulled via the OGGM shop, is used for comparison/calibration.


In [ ]:
# For demonstration CryoTempo-EOLIS data is provided in 2011 - 2020 and for the year 2015 only
ref_mb = dtcg_oggm.calibrator.get_geodetic_mb(gdir=gdir, dataset=datacube)
ref_mb

We define a **calibration matrix**: three calibration runs of the same model (`DailyTIModel`), each paired with a different **reference dataset** and **time window**:

- **Daily_Hugonnet**: calibrate with the geodetic mass balance from **Hugonnet et al. (2010–2020)**.  
- **Daily_Cryosat**: calibrate with **CryoTEMPO-EOLIS** over **2011–2020**.  
- **Daily_Cryosat_2015**: calibrate with **CryoTEMPO-EOLIS** over **2015** only.

Then `calibrator.calibrate(...)` runs the fits for all entries in the matrix. The result, `mb_models`, contains the **mass-balance models** with tuned parameters for each configuration.

In [ ]:
# Define which model should be calibrated with which data
dtcg_oggm.calibrator.set_model_matrix(
    name="Daily_Hugonnet",
    model=massbalance.DailyTIModel,
    geo_period="2010-01-01_2020-01-01",
    daily=True,
    source="Hugonnet",
    extra_kwargs={},
)
dtcg_oggm.calibrator.set_model_matrix(
    name="Daily_Cryosat",
    model=massbalance.DailyTIModel,
    geo_period="2011-01-01_2020-01-01",
    daily=True,
    source="CryoTEMPO-EOLIS",
    extra_kwargs={},
)
dtcg_oggm.calibrator.set_model_matrix(
    name="Daily_Cryosat_2015",
    model=massbalance.DailyTIModel,
    geo_period="2015-01-01_2016-01-01",
    daily=True,
    source="CryoTEMPO-EOLIS",
    extra_kwargs={},
)

# run_calibration
mb_models, _, _ = dtcg_oggm.calibrator.calibrate(
    model_matrix=dtcg_oggm.calibrator.model_matrix,
    gdir=gdir, ref_mb=ref_mb
)

Lets visualise the mass balance model outputs!

<mark>TODO: add description of output explaining 2015 anomaly</mark>

In [ ]:
fls = gdir.read_pickle('inversion_flowlines')
years = np.arange(2000, 2019)
for label, mbmod in mb_models.items():
    mb_ts = mbmod.get_specific_mb(fls=fls, year=years)
    plt.plot(years, mb_ts, label=label)
plt.ylabel('Specific MB (mm w.e.)')
plt.xlabel('Year')
plt.legend()

## L2 Datacube Creation & Export

We add these observation-informed, modeled mass-balance layers to the existing datacube via the `add_layer` function.

<mark>TODO: add metadata to the <a href="https://github.com/DTC-Glaciers/dtcg/blob/6fb73fee1429dece839ff2ef157e03f8adeca0ee/dtcg/datacube/metadata_mapping.yaml">YAML file</a></mark>

In [ ]:
# construct an xarray dataset with the modelled mass balance to add to the datacube
year_dts = [datetime(y, 1, 1, tzinfo=timezone.utc).timestamp() for y in years]
data_arrs = []
for label, mbmod in mb_models.items():
    data_arrs.append(
        xr.DataArray(
            mb_ts,
            dims=("t"),
            coords={"t": year_dts},
            name=label,
            attrs={"institution": "TBC",
                   "standard_name": "TBC",
                   "long_name": "TBC",
                   "references": "TBC",
                   "source": "TBC",
                   "units": "mm w.e.",
                   "comment": "TBC"
            }
        )
    )
ds = xr.merge(data_arrs)
ds.attrs.clear()    # clear dataset level attributes


# add the new dataset as L2 layer to the datacube
datacube_handler.add_layer(ds, "L2", overwrite=True)

The datacube is an xarray DataTree: **L1** holds observational glacier-domain variables, and **L2** holds the modelled data.

In [ ]:
datacube_handler.data_tree

[![Zarr logo](https://avatars.githubusercontent.com/u/35050297?s=96&v=4)](https://github.com/zarr-developers/geozarr-spec) Both **L1** and **L2** can be exported as a GeoZarr with the datacube's export function.

In [ ]:
from pathlib import Path
import tempfile

with tempfile.TemporaryDirectory(suffix=".zarr") as tmpdir:
    output_path = Path(tmpdir)
    datacube_handler.export(output_path)
    print(f"✅ GeoZarr exported to: {output_path}")
    items = sorted(p.name for p in output_path.iterdir())
    print("📂 Top-level contents:", ", ".join(items) or "(empty)")